#Bases de Dados Textuais

In [ ]:
!pip install git+https://github.com/GoloMarcos/FKTC/

from FakeNewsTextCollections import datasets

bases = datasets.load('datasets/')

  Cloning https://github.com/GoloMarcos/FKTC/ to /tmp/pip-req-build-4un2y3tt
  Running command git clone -q https://github.com/GoloMarcos/FKTC/ /tmp/pip-req-build-4un2y3tt
  Created wheel for FakeNewsTextCollections: filename=FakeNewsTextCollections-0.1.0-cp37-none-any.whl size=2058 sha256=b9b562e1de758869af31eee91e4c122bd67a2722cf3dd60e7035292a7845330f
  Stored in directory: /tmp/pip-ephem-wheel-cache-52_dul7a/wheels/16/c3/42/3e4f9da2d68ba20a9da361eec2ae670a4e48d4794e00ca647d
Successfully built FakeNewsTextCollections


# Train and Test


In [ ]:
from sklearn.model_selection import train_test_split

def train_test_split_pipeline(df_int, percent,column, f1, f2, f3, with_name, parcial_name=''):

  name = parcial_name + '_' + str(percent) + '_' + str(f1)

  if percent == 3 or percent == 5 or percent == 7:
    df_test = np.array(df_int[df_int.fold != f1][column].to_list()) 
    df_fold = np.array(df_int[df_int.fold == f1][column].to_list())
    train_percent = percent/10
    test_percent = 1 - train_percent
    df_train, discard = train_test_split(df_fold, test_size=test_percent, random_state=42)
    
  elif percent == 10:
    df_train = np.array(df_int[df_int.fold == f1][column].to_list())
    df_test = np.array(df_int[df_int.fold != f1][column].to_list())

  elif percent == 20:
    df_train = np.array(df_int[(df_int.fold == f1) | (df_int.fold == f2)][column].to_list())
    df_test = np.array(df_int[(df_int.fold != f1) & (df_int.fold != f2)][column].to_list())
    name += '-' + str(f2)

  elif percent == 30:
    df_train = np.array(df_int[(df_int.fold == f1) | (df_int.fold == f2) | (df_int.fold == f3)][column].to_list())
    df_test = np.array(df_int[(df_int.fold != f1) & (df_int.fold != f2) & (df_int.fold != f3)][column].to_list())
    name += '-' + str(f2) + '-' + str(f3)

  if with_name:
    return df_train, df_test, name
  else:
    return df_train, df_test

# One Class Classification

In [ ]:
def init_metrics():
  metricas = {
    'precisao' : [],
    'revocacao' : [],
    'f1-score' : [],
    'auc_roc' : [],
    'acuracia' : [],
    'tempo' : []
  }

  return metricas

In [ ]:
def save_values(metricas, values):
  metricas['precisao'].append(values['precisão'])
  metricas['revocacao'].append(values['revocação'])
  metricas['f1-score'].append(values['f1-score'])
  metricas['auc_roc'].append(values['auc_roc'])
  metricas['acuracia'].append(values['acurácia'])
  metricas['tempo'].append(values['tempo'])

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import time
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
from gc import collect


def evaluation_one_class(preds_interest, preds_outliers):
  y_true = [1]*len(preds_interest) + [-1]*len(preds_outliers)
  y_pred = list(preds_interest)+list(preds_outliers)
  return classification_report(y_true, y_pred, output_dict=True)

def evaluate_model(X_train, X_test, X_outlier, model):
  
  one_class_classifier = model.fit(X_train)

  Y_pred_interest = one_class_classifier.predict(X_test)
  
  Y_pred_ruido = one_class_classifier.predict(X_outlier)

  score_interest = one_class_classifier.decision_function(X_test)

  score_outlier = one_class_classifier.decision_function(X_outlier)

  y_true = np.array([1] * len(X_test) + [-1] * len(X_outlier))

  fpr, tpr, limiares = roc_curve(y_true, np.concatenate([score_interest,score_outlier]))

  dic = evaluation_one_class(Y_pred_interest, Y_pred_ruido)

  metricas = {}
  metricas['precisão'] = dic['1']['precision']
  metricas['revocação'] = dic['1']['recall']
  metricas['f1-score'] = dic['1']['f1-score']
  metricas['auc_roc'] = roc_auc_score(y_true, np.concatenate([score_interest,score_outlier]))
  metricas['acurácia'] = dic['accuracy']

  return metricas,fpr,tpr

def escreve(metricas, file_name, line_parameters, folds, caminho):
  
  path = caminho
  if not Path(path + file_name).is_file():
    arquivo = open(path + file_name,'w')
    string = 'Parametros'
    
    for metrica in metricas.keys():
      string+= ';' + metrica + '-mean;' + metrica + '-std'
    string+= '\n'
    
    arquivo.write(string)
    arquivo.close()
  
  arquivo = open(path + file_name,'a')
  string = line_parameters

  for metrica in metricas.keys():
    string += ';' + str(np.mean(metricas[metrica])) + ';' + str(np.std(metricas[metrica])) 
    
  string+= '\n'
  arquivo.write(string)
  arquivo.close()


In [ ]:
import matplotlib.pyplot as plt
def save_roc_curve(fpr,tpr,auc_roc, name):
  path = ''
  name += '.pdf'
  fig = plt.figure()
  plt.plot(fpr, tpr, label='AUC ROC = %0.4f' % auc_roc)
  plt.plot([0, 1], [0, 1], 'k--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('Taxa de Falso Positivo (TFP)')
  plt.ylabel('Taxa de Verdadeiro Positivo (TVP)')
  plt.title('Curva ROC')
  plt.legend(loc="lower right")
  plt.savefig(path+name)   
  plt.close(fig)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

def plot_TSNE(X_int, X_outlier, name):
  
  path = '/content/drive/MyDrive/USP/projeto/artigos/FakeNews/TSNEs/'
  name += '.pdf'

  X = np.concatenate((X_outlier,X_int))

  X_embedded = TSNE(n_components=2).fit_transform(X)

  df1 = pd.DataFrame(X_embedded)
  df1['Classe'] = np.array(['Não Interesse'] * len(X_outlier) + ['Interesse'] * len(X_int))
  df1 = df1.dropna()

  g = sns.scatterplot(x=0,y=1,data=df1, hue="Classe", legend=True)
  g.set(xlabel=None)
  g.set(ylabel=None)
  plt.savefig(path+name)
  plt.close()

# One Clas SVDD

importar sklearn normal antes de rodar esses trechos de código para o sklearn como o SVDD

In [ ]:
!gdown --id 1HfSoQ7SSsdmIVzVk1qqWyoP_URR4a5Bi

Downloading...
From: https://drive.google.com/uc?id=1HfSoQ7SSsdmIVzVk1qqWyoP_URR4a5Bi
To: /content/scikit-learn.tar.gz
178MB [00:03, 48.8MB/s]


In [ ]:
!tar -xzvf scikit-learn.tar.gz

In [ ]:
cd scikit-learn/

/content/scikit-learn


In [ ]:
!mv sklearn sklearn_svdd

In [ ]:
from sklearn_svdd.svm import SVDD
modelos = {
    'SVDD_RBF_0.001_scale' : SVDD(kernel='rbf', nu = 0.001, gamma='scale'),
    'SVDD_RBF_0.01_scale' : SVDD(kernel='rbf', nu = 0.01, gamma='scale'),
    'SVDD_RBF_0.05_scale' : SVDD(kernel='rbf', nu=0.05, gamma='scale'),
    'SVDD_RBF_0.1_scale' : SVDD(kernel='rbf', nu = 0.1, gamma='scale'),
    'SVDD_RBF_0.2_scale' : SVDD(kernel='rbf', nu = 0.2, gamma='scale'),
    'SVDD_RBF_0.3_scale' : SVDD(kernel='rbf', nu = 0.3, gamma='scale'),
    'SVDD_RBF_0.4_scale' : SVDD(kernel='rbf', nu = 0.4, gamma='scale'),
    'SVDD_RBF_0.5_scale' : SVDD(kernel='rbf', nu = 0.5, gamma='scale'),
    'SVDD_RBF_0.6_scale' : SVDD(kernel='rbf', nu = 0.6, gamma='scale'),
    'SVDD_RBF_0.7_scale' : SVDD(kernel='rbf', nu = 0.7, gamma='scale'),
    'SVDD_RBF_0.8_scale' : SVDD(kernel='rbf', nu = 0.8, gamma='scale'),
    'SVDD_RBF_0.9_scale' : SVDD(kernel='rbf', nu = 0.9, gamma='scale')
}

In [ ]:
cd ..

/content


# Bag-of-Words

## Code


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


def hasNumbers(inputString):
 return any(char.isdigit() for char in inputString)

class MyTokenizer:
    def __init__(self, language):
        self.wnl = WordNetLemmatizer()
        if language == 'english':
          self.STOPWORDS = nltk.corpus.stopwords.words('english')
          self.stemmer = nltk.stem.SnowballStemmer('english')

        if language == 'spanish':
          self.STOPWORDS = nltk.corpus.stopwords.words('spanish')
          self.stemmer = nltk.stem.SnowballStemmer('spanish')

        if language == 'portuguese':
          self.STOPWORDS = nltk.corpus.stopwords.words('portuguese')
          self.stemmer = nltk.stem.SnowballStemmer('portuguese')

        if language == 'multilingual':
          stop_words_pt = nltk.corpus.stopwords.words('portuguese')
          stop_words_es = nltk.corpus.stopwords.words('spanish')
          stop_words_en = nltk.corpus.stopwords.words('english')
          self.STOPWORDS = set(nltk.corpus.stopwords.words('spanish')).union(set(nltk.corpus.stopwords.words('portuguese'))).union(set(nltk.corpus.stopwords.words('english')))
          self.stemmer = nltk.stem.SnowballStemmer('english')

    def __call__(self, doc):
        L1 = [t for t in word_tokenize(doc)]
        L2 = []
        for token in L1:
          if token not in self.STOPWORDS and token.isnumeric()==False and len(token) > 2 and hasNumbers(token)==False: L2.append(token)
        L3 = [self.stemmer.stem(self.wnl.lemmatize(t)) for t in L2]
        return L3
        
def tipo_peso_termo(tipo_bow):
  if tipo_bow == 'term-frequency':
    vectorizer = CountVectorizer(strip_accents='ascii', stop_words='english', ngram_range=(1,1), min_df=1, tokenizer=MyTokenizer('multilingual'))
  if tipo_bow == 'term-frequency-IDF':
    vectorizer = TfidfVectorizer(strip_accents='ascii', stop_words='english', ngram_range=(1,1), min_df=1, tokenizer=MyTokenizer('multilingual'))
  if tipo_bow == 'binary':
    vectorizer = CountVectorizer(strip_accents='ascii', stop_words='english', ngram_range=(1,1), min_df=1, tokenizer=MyTokenizer('multilingual'), binary=True)
  
  return vectorizer

## Evaluation

In [ ]:
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

lista_pesos_termo = ['term-frequency-IDF', 'term-frequency', 'binary']

caminho = './resultados/'

percents = [3,5,7,10]

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)
  
  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out.text.to_list())

  for peso_de_termo in tqdm(lista_pesos_termo):
    print('No peso de termo: ' + peso_de_termo)
    vectorizer = tipo_peso_termo(peso_de_termo)

    for percent in tqdm(percents):
      print('No percentual: ' + str(percent))

      lista_reps = list()

      for fold in tqdm(folds):
        print('No fold ' + str(fold))

        f1 = fold
        f2 = (fold+1) % len(folds)
        f3 = (fold+2) % len(folds)
        
        df_train, df_test = train_test_split_pipeline(df_int, percent,'text', f1, f2, f3, False)

        vectorizer.fit(df_train)

        X_train = vectorizer.transform(df_train)
        X_test = vectorizer.transform(df_test)
        X_outlier = vectorizer.transform(df_outlier)

        reps = (X_train,X_test,X_outlier)

        lista_reps.append(reps)
        
        del df_train
        del df_test
        del X_train
        del X_test
        del X_outlier
        collect()

      for modelo in tqdm(modelos):
        print('No modelo: ' + modelo)
        metricas = init_metrics()
     
        for reps in lista_reps:
          start = time.time()
          values,fpr,tpr = evaluate_model(reps[0].toarray(), reps[1].toarray(), reps[2].toarray(), modelos[modelo])
          end = time.time()
          temp = end - start

          values['tempo'] = temp
          
          save_values(metricas,values)

        nome_arquivo = base + '_BoW_' + peso_de_termo + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

        parametros_linha = modelo

        escreve(metricas, nome_arquivo, parametros_linha, len(folds), caminho)
      
      del lista_reps
      collect()
      
  del df
  del df_int
  del df_out
  del df_outlier
  collect()

## TSNE

In [ ]:
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

lista_pesos_termo = ['term-frequency-IDF', 'term-frequency', 'binary']

for base in bases.keys():
  df = bases[base]
  print('Na base: ' + base)
  
  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out.text.to_list())

  for peso_de_termo in tqdm(lista_pesos_termo):
    print('No peso de termo: ' + peso_de_termo)
    vectorizer = tipo_peso_termo(peso_de_termo)

    vectorizer.fit(df_int.text.to_list())

    X_int = vectorizer.transform(df_int.text.to_list())
    X_outlier = vectorizer.transform(df_outlier)
    
    name = base + '_' + peso_de_termo 

    plot_TSNE(X_int.toarray(), X_outlier.toarray(), name)

# Word-Embeddings

## Evaluation

In [ ]:
caminho = './resultados/'

percents = [3,5,7,10]

from tqdm.notebook import tqdm

word_embeddings = ['DistilBERT Multilingua']

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):
    print('Na word-embedding: ' + str(word_embedding))

    df_outlier = np.array(df_out[word_embedding].to_list())

    for percent in tqdm(percents):
      print('No percentual: ' + str(percent))

      lista_reps = []

      for fold in tqdm(folds):
        print('No fold ' + str(fold))

        f1 = fold
        f2 = (fold+1) % len(folds)
        f3 = (fold+2) % len(folds)
        
        df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3, False)
        
        reps = (df_train,df_test,df_outlier)

        lista_reps.append(reps)
        
        del df_train
        del df_test
        collect()

      for modelo in tqdm(modelos):
        print('No modelo: ' + modelo)

        metricas = init_metrics()

        for reps in lista_reps:
          start = time.time()
          values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
          end = time.time()
          temp = end - start
          
          values['tempo'] = temp

          save_values(metricas,values)

        nome_arquivo = base +'_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

        parametros_linha = modelo 
        
        escreve(metricas,nome_arquivo,parametros_linha, len(folds), caminho)

      del lista_reps
      collect()
    
    del df_outlier
    collect()

  del df
  del df_int
  del df_out
  collect()

## TSNE

In [ ]:
from tqdm.notebook import tqdm

word_embeddings = ['DistilBERT Multilingua']

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_inlier = np.array(df_int['DistilBERT Multilingua'].to_list())
  
  df_outlier = np.array(df_out['DistilBERT Multilingua'].to_list())
      
  name = 'DistilBERT/tsne_' + base 

  plot_TSNE(df_inlier, df_outlier, name)

# Density Information

## Code


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from tqdm.notebook import tqdm

def informacao_de_densidade(lista_grupos,df_train,df_test,df_outlier):
  L_X_train = []
  L_X_test = []
  L_X_outlier = []

  for quantidade_de_grupos in tqdm(lista_grupos):
    
    kmeans = KMeans(n_clusters=quantidade_de_grupos, random_state=0).fit(df_train)
    
    X_train_temp = silhouette_samples(df_train, kmeans.labels_).reshape(len(df_train),1)
    L_X_train.append(X_train_temp)

    X_test_temp = silhouette_samples(df_test, kmeans.predict(df_test)).reshape(len(df_test),1)
    L_X_test.append(X_test_temp)

    X_outlier_temp = silhouette_samples(df_outlier, kmeans.predict(df_outlier)).reshape(len(df_outlier),1)
    L_X_outlier.append(X_outlier_temp)

   
  return np.concatenate(L_X_train,axis=1), np.concatenate(L_X_test,axis=1), np.concatenate(L_X_outlier,axis=1)


## Evaluation

In [ ]:
matriz_grupos = [[2,4,6,8,10],
                 [3,5,7,9],
                 [2,3,4,5,6,7,8,9,10]
                ] 

caminho = './resultados/'

percents = [3,5,7,10]

word_embeddings = ['DistilBERT Multilingua']
  
for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):
    print('Na word-embedding: ' + str(word_embedding))

    df_outlier = np.array(df_out[word_embedding].to_list())

    for percent in tqdm(percents):
      print('No percentual: ' + str(percent))

      for lista_grupos in tqdm(matriz_grupos):
        
        lista_reps = []
        
        for fold in tqdm(folds):
          print('No fold ' + str(fold))

          f1 = fold
          f2 = (fold+1) % len(folds)
          f3 = (fold+2) % len(folds)

          df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,False)

          X_train,X_test,X_outlier = informacao_de_densidade(lista_grupos,df_train,df_test,df_outlier)

          reps = (X_train,X_test,X_outlier)

          lista_reps.append(reps)
          
          del df_train
          del df_test
          del X_train
          del X_test
          del X_outlier
          collect()

        for modelo in tqdm(modelos):
          print('No modelo: ' + modelo)

          metricas = init_metrics()

          for reps in lista_reps:
            start = time.time()
            values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
            end = time.time()
            temp = end - start
            
            values['tempo'] = temp

            save_values(metricas,values)

          nome_arquivo = base + '_InfDensidade_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

          parametros_linha = modelo + '_' + str(lista_grupos)

          escreve(metricas, nome_arquivo,parametros_linha, len(folds), caminho)
        
        del lista_reps
        collect()
    
    del df_outlier
    collect()

  del df
  del df_int
  del df_out
  collect()


## TSNE

In [ ]:
dic = {
    'fcn' : { 3 : [3, 5, 7, 9],
              5 : [2, 4, 6, 8, 10],
              7 : [3, 5, 7, 9],
              10 : [2, 4, 6, 8, 10],
    },
    'fnn' : { 3 : [2, 4, 6, 8, 10],
              5 : [2, 4, 6, 8, 10],
              7 : [3, 5, 7, 9],
              10 : [2, 4, 6, 8, 10],
    },
    'fakebr' : { 3 : [3, 5, 7, 9],
                5 : [2, 4, 6, 8, 10],
                7 : [2, 3, 4, 5, 6, 7, 8, 9, 10],
                10 : [2, 3, 4, 5, 6, 7, 8, 9, 10],
    }
}

In [ ]:
word_embedding = 'DistilBERT Multilingua'
percents = [3,5,7,10]

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out[word_embedding].to_list())

  for percent in tqdm(percents):
    print('No percentual: ' + str(percent))

    lista_grupos = dic[base][percent]

    for fold in tqdm(folds):
      print('No fold ' + str(fold))

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)

      df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,False)

      X_train,X_test,X_outlier = informacao_de_densidade(lista_grupos,df_train,df_test,df_outlier)

      name = 'Densidade/' + base + '/' + str(percent) + '/tsne_fold' + str(fold)

      plot_TSNE(np.concatenate((X_train,X_test)),X_outlier, name)

# LIWK

## Evaluation


In [ ]:
caminho = './resultados/'

percents = [3,5,7,10]

from tqdm.notebook import tqdm

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out['features_normalized'].to_list())

  for percent in tqdm(percents):
    print('No percentual: ' + str(percent))

    lista_reps = []

    for fold in tqdm(folds):
      print('No fold ' + str(fold))

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)
      
      df_train,df_test = train_test_split_pipeline(df_int, percent,'features_normalized', f1,f2,f3,False)
    
      reps = (df_train,df_test,df_outlier)

      lista_reps.append(reps)
      
      del df_train
      del df_test
      collect()
    
    for modelo in tqdm(modelos):
      print('No modelo: ' + modelo)

      metricas = init_metrics()    
        
      for reps in lista_reps:

        start = time.time()
        values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
        end = time.time()
        temp = end - start
        values['tempo'] = temp
        save_values(metricas,values)

      nome_arquivo = base +'_FeaturesNormalized_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

      parametros_linha = modelo 
      
      escreve(metricas,nome_arquivo,parametros_linha, len(folds), caminho)
    
    del lista_reps
    collect()  
    
  del df
  del df_int
  del df_out
  del df_outlier
  collect()

## TSNE

In [ ]:
from tqdm.notebook import tqdm

for base in tqdm(bases):
  df = bases[base]
  print('Na base: ' + base)

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out['features_normalized'].to_list())

  df_inlier = np.array(df_int['features_normalized'].to_list())

  name = 'LIWK/tsne_' + base 

  plot_TSNE(df_inlier, df_outlier, name)  

# Autoencoder Unimodal (modalidade = Word-Embedding)

## Code

In [ ]:
import tensorflow
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model, save_model

def autoencoder(arquitetura, len_entrada):

  encoder_inputs = Input(shape=(len_entrada, ), name='Entrada_Encoder') # entrada
  
  if len(arquitetura) == 3:
    camada1 = Dense(arquitetura[0], activation="linear")(encoder_inputs) # camada 1

    camada2 = Dense(arquitetura[1], activation="linear")(camada1) # camada 2
    
    encoded = Dense(arquitetura[2], activation="linear")(camada2) # code

    camada3 = Dense(arquitetura[1], activation="linear")(encoded) # camada 3

    camada4 = Dense(arquitetura[0], activation="linear")(camada3) # camada 4

    decoder_output = Dense(len_entrada, activation="linear")(camada4) # saida
  
  if len(arquitetura) == 2:
    camada1 = Dense(arquitetura[0], activation="linear")(encoder_inputs) # camada 1
    
    encoded = Dense(arquitetura[1], activation="linear")(camada1) # code

    camada2 = Dense(arquitetura[0], activation="linear")(encoded) # camada 2

    decoder_output = Dense(len_entrada, activation="linear")(camada2) # saida
  
  if len(arquitetura) == 1:
    encoded = Dense(arquitetura[0], activation="linear")(encoder_inputs) # code

    decoder_output = Dense(len_entrada, activation="linear")(encoded) # saida


  encoder = Model(encoder_inputs,  encoded)

  autoencoder = Model(encoder_inputs, decoder_output)

  autoencoder.compile(optimizer=tensorflow.keras.optimizers.Adam(), loss='mse')

  return autoencoder, encoder

## Evaluation

In [ ]:
from tqdm.notebook import tqdm

epocas = [5,10,50]

arquiteturas = [[384,256],   # entrada tem tamanho do vetor da word-embedding
                [256,128],   # aqui são definidas a segunda camada e/ou a camada codificada
                [256]
                ] 

caminho_resultado = './resultados/'

percents = [3,5,7,10]

caminho = './modelos/AE/' # caminho para salvar os modelos de rede neurais aprendidos

word_embeddings = ['DistilBERT Multilingua']
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):
    print('Na word-embedding: ' + str(word_embedding))

    df_outlier = np.array(df_out[word_embedding].to_list())

    for percent in tqdm(percents):
      print('No percentual: ' + str(percent))

      for epoca in tqdm(epocas):

        for arq in tqdm(arquiteturas):
            
          name_arq = str(arq).replace('[', '(')
          name_arq = name_arq.replace(']',')')

          lista_reps = []

          for fold in tqdm(folds):
            print('No fold ' + str(fold))

            f1 = fold
            f2 = (fold+1) % len(folds)
            f3 = (fold+2) % len(folds)

            parcial_name = base + '_' + word_embedding + '_' + str(epoca) + '_' + str(name_arq) 

            df_train,df_test,nome_arq = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,True,parcial_name)
            
            ae,encoder = autoencoder(arq,len(df_train[0]))

            ae.fit(df_train, df_train, epochs=epoca, batch_size=32, verbose=0)

            save_model(ae,caminho + nome_arq + '_autoencoder')
            save_model(encoder,caminho + nome_arq + '_encoder')

            # para carregar = "load_model('path/to/location')"

            X_train = encoder.predict(df_train)
            X_test = encoder.predict(df_test)
            X_outlier = encoder.predict(df_outlier)

            reps = (X_train,X_test,X_outlier)

            lista_reps.append(reps)

            del df_train
            del df_test
            del X_train
            del X_test
            del X_outlier
            collect()
              
          for modelo in tqdm(modelos):
            print('No modelo: ' + modelo)

            metricas = init_metrics()
            
            for reps in lista_reps:
              start = time.time()
              values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
              end = time.time()
              temp = end - start
              
              values['tempo'] = temp

              save_values(metricas,values)

            nome_arquivo = base + '_Autoencoder_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

            parametros_linha = modelo + '_' + str(epoca) + '_' + str(arq)

            escreve(metricas,nome_arquivo,parametros_linha, len(folds) , caminho_resultado)
          
          del lista_reps
          collect()

    del df_outlier
    collect()

  del df
  del df_int
  del df_out
  collect()

## TSNE

In [ ]:
dic = {
    'fcn' : { 3 : (50,[256, 128]),
              5 : (50,[256, 128]),
              7 : (50,[256, 128]),
              10 : (50,[256]),
    },
    'fnn' : { 3 : (50,[256, 128]),
              5 : (50,[256, 128]),
              7 : (50,[256, 128]),
              10 : (50,[256, 128]),
    },
    'fakebr' : { 3 : (50,[256, 128]),
              5 : (50,[256, 128]),
              7 : (50,[256, 128]),
              10 : (50,[256, 128]),
    },
}

In [ ]:
from tqdm.notebook import tqdm

percents = [3,5,7,10]

word_embedding = 'DistilBERT Multilingua'
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out[word_embedding].to_list())

  for percent in tqdm(percents):
    print('No percentual: ' + str(percent))

    epoca = dic[base][percent][0]
    arq = dic[base][percent][1]

    for fold in tqdm(folds):
      print('No fold ' + str(fold))

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)

      df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3, False)
      
      ae,encoder = autoencoder(arq,len(df_train[0]))

      ae.fit(df_train, df_train, epochs=epoca, batch_size=32, verbose=0)

      X_train = encoder.predict(df_train)
      X_test = encoder.predict(df_test)
      X_outlier = encoder.predict(df_outlier)

      name = 'AE/' + base + '/' + str(percent) + '/tsne_fold' + str(f1) 

      plot_TSNE(np.concatenate((X_train,X_test)), X_outlier, name)

# Variational Autoencoder Unimodal 

## Code

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model, save_model

class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z."""

  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(keras.Model):
  def __init__(self, encoder, decoder, factor_multiply,  **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.factor_multiply = factor_multiply

  def train_step(self, data):
    if isinstance(data, tuple):
      data = data[0]
    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = encoder(data)
      reconstruction = decoder(z)
      reconstruction_loss = tf.reduce_mean(
          keras.losses.mean_squared_error(data, reconstruction)
      )
      reconstruction_loss *= self.factor_multiply
      kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
      kl_loss = tf.reduce_mean(kl_loss)
      kl_loss *= -0.5
      total_loss = reconstruction_loss + kl_loss


    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {
      "loss": total_loss,
      "reconstruction_loss": reconstruction_loss,
      "kl_loss": kl_loss,
    }


In [ ]:
def Encoder_VAE(arquitetura,input_dim):

  encoder_inputs = keras.Input(shape=(input_dim,), name='Entrada_Encoder') 

  if len(arquitetura) == 3:
    camada1 = Dense(arquitetura[0], activation="linear")(encoder_inputs) # camada 1

    camada2 = Dense(arquitetura[1], activation="linear")(camada1) # camada 2
    
    z_mean = layers.Dense(arquitetura[2], name="Z_mean")(camada2)
    z_log_var = layers.Dense(arquitetura[2], name="Z_log_var")(camada2)
    z = Sampling()([z_mean, z_log_var])

  if len(arquitetura) == 2:
    camada1 = Dense(arquitetura[0], activation="linear")(encoder_inputs) # camada 1
    
    z_mean = layers.Dense(arquitetura[1], name="Z_mean")(camada1)
    z_log_var = layers.Dense(arquitetura[1], name="Z_log_var")(camada1)
    z = Sampling()([z_mean, z_log_var])
  
  if len(arquitetura) == 1:
    z_mean = layers.Dense(arquitetura[0], name="Z_mean")(encoder_inputs)
    z_log_var = layers.Dense(arquitetura[0], name="Z_log_var")(encoder_inputs)
    z = Sampling()([z_mean, z_log_var])


  encoder = keras.Model([encoder_inputs], [z_mean, z_log_var, z], name="Encoder")

  return encoder

def Decoder_VAE(arquitetura,output_dim):
  
  latent_inputs = keras.Input(shape=(arquitetura[(len(arquitetura)-1)],),name='Entrada_Decoder')

  if len(arquitetura) == 3:
    camada1 = Dense(arquitetura[1], activation="linear")(latent_inputs) # camada 1

    camada2 = Dense(arquitetura[0], activation="linear")(camada1) # camada 2
    
    decoder_outputs = Dense(output_dim, activation="linear")(camada2) #  saida

  if len(arquitetura) == 2:
    camada1 = Dense(arquitetura[0], activation="linear")(latent_inputs) # camada 1

    decoder_outputs = Dense(output_dim, activation="linear")(camada1) # saida
  
  if len(arquitetura) == 1:
    decoder_outputs = Dense(output_dim, activation="linear")(latent_inputs) # camada 2
  
  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

  return decoder

In [ ]:
def VariationalAutoencoder(arquitetura,input_dim):    

  encoder = Encoder_VAE(arquitetura,input_dim)

  decoder = Decoder_VAE(arquitetura,input_dim)
          
  vae = VAE(encoder, decoder, input_dim)

  vae.compile(optimizer=keras.optimizers.Adam())

  return vae, encoder, decoder

## Evaluation

In [ ]:
from tqdm.notebook import tqdm

epocas = [5,10,50]

arquiteturas = [[384,256],   # entrada tem tamanho do vetor da word-embedding
                [256,128],   # aqui são definidas a segunda camada e/ou a camada codificada
                [256]
                ] 

caminho_resultado = './resultados/'

percents = [3,5,7,10]

caminho = './modelos/VAE/' # caminho para salvar os modelos de rede neurais aprendidos


word_embeddings = ['DistilBERT Multilingua']
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):
    print('Na word-embedding: ' + str(word_embedding))

    df_outlier = np.array(df_out[word_embedding].to_list())

    for percent in tqdm(percents):
      print('No percentual: ' + str(percent))
    
      for epoca in tqdm(epocas):

        for arq in tqdm(arquiteturas):
            
          name_arq = str(arq).replace('[', '(')
          name_arq = name_arq.replace(']',')')

          lista_reps = []
          
          for fold in tqdm(folds):
            print('No fold ' + str(fold))

            f1 = fold
            f2 = (fold+1) % len(folds)
            f3 = (fold+2) % len(folds)
            
            parcial_name = base + '_' + word_embedding + '_' + str(epoca) + '_' + str(name_arq) 
            
            df_train,df_test,nome_arq = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,True,parcial_name)
            
            vae,encoder,decoder = VariationalAutoencoder(arq,len(df_train[0]))

            vae.fit(df_train, df_train, epochs=epoca, batch_size=32, verbose=0)
            
            save_model(encoder,caminho + nome_arq + '_encoder')
            save_model(decoder,caminho + nome_arq + '_decoder')

            # para carregar = "load_model('path/to/location')""

            X_train,_,_ = encoder.predict(df_train)
            X_test,_,_ = encoder.predict(df_test)
            X_outlier,_,_ = encoder.predict(df_outlier)
            
            reps = (X_train,X_test,X_outlier)

            lista_reps.append(reps)

            del df_train
            del df_test
            del X_train
            del X_test
            del X_outlier
            collect()

          for modelo in tqdm(modelos):
            print('No modelo: ' + modelo)

            metricas = init_metrics()

            for reps in lista_reps:
              start = time.time()
              values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
              end = time.time()
              temp = end - start
              
              values['tempo'] = temp

              save_values(metricas,values)
            
            nome_arquivo = base + '_VAE_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

            parametros_linha = modelo + '_' + str(epoca) + '_' + str(arq)
            
            escreve(metricas,nome_arquivo,parametros_linha,len(folds), caminho_resultado)

          del lista_reps
          collect()

    del df_outlier
    collect()

  del df
  del df_int
  del df_out
  collect()

## TSNE

In [ ]:
dic = {
    'fcn' : { 3 : (10,[384, 256]),
              5 : (10,[384, 256]),
              7 : (5,[384, 256]),
              10 : (5,[384, 256]),
    },
    'fnn' : { 3 : (10,[384, 256]),
              5 : (10,[384, 256]),
              7 : (10,[384, 256]),
              10 : (5,[384, 256]),
    },
    'fakebr' : { 3 : (5,[384, 256]),
              5 : (5,[384, 256]),
              7 : (5,[384, 256]),
              10 : (5,[384, 256]),
    },
}


from tqdm.notebook import tqdm

percents = [3,5,7,10]

word_embedding = 'DistilBERT Multilingua'
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out[word_embedding].to_list())

  for percent in tqdm(percents):
    print('No percentual: ' + str(percent))

    epoca = dic[base][percent][0]
    arq = dic[base][percent][1]

    for fold in tqdm(folds):
      print('No fold ' + str(fold))

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)

      df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3, False)
      
      vae,encoder,decoder = VariationalAutoencoder(arq,len(df_train[0]))

      vae.fit(df_train, df_train, epochs=epoca, batch_size=32, verbose=0)

      X_train,_,_ = encoder.predict(df_train)
      X_test,_,_ = encoder.predict(df_test)
      X_outlier,_,_ = encoder.predict(df_outlier)

      name = 'VAE/' + base + '/' + str(percent) + '/tsne_fold' + str(f1) 

      plot_TSNE(np.concatenate((X_train,X_test)), X_outlier, name)

# Multimodal Variational Autoencoder (MVAE)

## Code

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, concatenate, multiply, average, subtract, add
from tensorflow.keras.models import Model, load_model, save_model

class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z."""

  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class MVAE(keras.Model):
  def __init__(self, encoder, decoder, factor_multiply_embedding ,factor_multiply_density,  **kwargs):
    super(MVAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.factor_multiply_embedding = factor_multiply_embedding
    self.factor_multiply_density = factor_multiply_density

  def train_step(self, data):

    if isinstance(data, tuple):
      data = data[0]
    with tf.GradientTape() as tape:
      
      z_mean, z_log_var, z = self.encoder((data[0],data[1]))

      reconstruction = self.decoder(z)

      embedding_loss = tf.reduce_mean(
          keras.losses.mean_squared_error(data[0], reconstruction[0])
      )
      
      embedding_loss *= self.factor_multiply_embedding

      density_loss = tf.reduce_mean(
          keras.losses.mean_squared_error(data[1], reconstruction[1])
      )
      
      density_loss *= self.factor_multiply_density

      kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
      kl_loss = tf.reduce_mean(kl_loss)
      kl_loss *= -0.5
      total_loss = embedding_loss + density_loss + kl_loss


    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    return {
      "tota loss": total_loss,
      "embedding loss": embedding_loss,
      "denisty loss": density_loss,
      "kl loss": kl_loss,
    }


In [ ]:
def Encoder_MVAE(arquitetura,embedding_dim, density_dim, operator):

  embedding_inputs = keras.Input(shape=(embedding_dim,), name='Entrada_Embedding_Encoder')
  density_inputs = keras.Input(shape=(density_dim,), name='Entrada_Density_Encoder') 

  l1 = Dense(np.max([embedding_dim,density_dim]), activation='linear')(embedding_inputs)
  l2 = Dense(np.max([embedding_dim,density_dim]), activation='linear')(density_inputs)

  fusion = None
  if operator=='concatenate': fusion = concatenate([l1,l2])
  if operator=='multiply': fusion = multiply([l1,l2])
  if operator=='average': fusion = average([l1,l2])
  if operator=='subtract': fusion = subtract([l1,l2])
  if operator=='add': fusion = add([l1,l2])

  
  if len(arquitetura) == 3:
    camada1 = Dense(arquitetura[0], activation="linear")(fusion) # camada 1

    camada2 = Dense(arquitetura[1], activation="linear")(camada1) # camada 2
    
    z_mean = layers.Dense(arquitetura[2], name="Z_mean")(camada2)
    z_log_var = layers.Dense(arquitetura[2], name="Z_log_var")(camada2)
    z = Sampling()([z_mean, z_log_var])

  if len(arquitetura) == 2:
    camada1 = Dense(arquitetura[0], activation="linear")(fusion) # camada 1
    
    z_mean = layers.Dense(arquitetura[1], name="Z_mean")(camada1)
    z_log_var = layers.Dense(arquitetura[1], name="Z_log_var")(camada1)
    z = Sampling()([z_mean, z_log_var])
  
  if len(arquitetura) == 1:
    z_mean = layers.Dense(arquitetura[0], name="Z_mean")(fusion)
    z_log_var = layers.Dense(arquitetura[0], name="Z_log_var")(fusion)
    z = Sampling()([z_mean, z_log_var])


  encoder = keras.Model([embedding_inputs,density_inputs], [z_mean, z_log_var, z], name="Encoder")

  return encoder

def Decoder_MVAE(arquitetura, embedding_dim, density_dim):
  
  latent_inputs = keras.Input(shape=(arquitetura[(len(arquitetura)-1)],),name='Entrada_Decoder')

  if len(arquitetura) == 3:
    camada1 = Dense(arquitetura[1], activation="linear")(latent_inputs) # camada 1

    camada2 = Dense(arquitetura[0], activation="linear")(camada1) # camada 2
    
    embedding_outputs = Dense(embedding_dim, activation="linear")(camada2) #  saida embedding

    density_outputs = Dense(density_dim, activation="linear")(camada2) #  saida density

  if len(arquitetura) == 2:
    camada1 = Dense(arquitetura[0], activation="linear")(latent_inputs) # camada 1

    embedding_outputs = Dense(embedding_dim, activation="linear")(camada1) #  saida embedding

    density_outputs = Dense(density_dim, activation="linear")(camada1) #  saida density
  
  if len(arquitetura) == 1:
    embedding_outputs = Dense(embedding_dim, activation="linear")(latent_inputs) #  saida embedding

    density_outputs = Dense(density_dim, activation="linear")(latent_inputs) #  saida density
  
  decoder = keras.Model(latent_inputs, [embedding_outputs,density_outputs], name="decoder")

  return decoder

In [ ]:
def MultimodalVAE(arquitetura,embedding_dim,density_dim,operator):    

  encoder = Encoder_MVAE(arquitetura, embedding_dim, density_dim, operator)

  decoder = Decoder_MVAE(arquitetura, embedding_dim, density_dim)
          
  mvae = MVAE(encoder, decoder, embedding_dim, density_dim)

  mvae.compile(optimizer=keras.optimizers.Adam())

  return mvae, encoder, decoder

## Evaluation MVAE-FakeNews

In [ ]:
from tqdm.notebook import tqdm

epocas = [5,10,50]

arquiteturas = [[384,256],   # entrada tem tamanho do vetor da word-embedding
                [256,128],   # aqui são definidas a segunda camada e/ou a camada codificada
                [256]
                ] 

caminho_resultado = './resultados/'

percents = [3,5,7,10]

matriz_grupos = [[2,4,6,8,10],
                 [3,5,7,9],
                 [2,3,4,5,6,7,8,9,10]
                ] 

operadores = ['concatenate','multiply','average','subtract','add']

caminho = './modelos/MVAE/' # caminho para salvar os modelos de rede neurais aprendidos

word_embeddings = ['DistilBERT Multilingua']
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):
    print('Na word-embedding: ' + str(word_embedding))

    df_outlier = np.array(df_out[word_embedding].to_list())

    for percent in tqdm(percents):
    
        for epoca in tqdm(epocas):

            for arq in tqdm(arquiteturas):
                
                name_arq = str(arq).replace('[', '(')
                name_arq = name_arq.replace(']',')')

                for lista_grupos in tqdm(matriz_grupos):
                    
                    name_lista_grupos = str(lista_grupos).replace('[', '(')
                    name_lista_grupos = name_lista_grupos.replace(']', ')')

                    for operador in tqdm(operadores):

                        lista_reps = []

                        for fold in tqdm(folds):

                            f1 = fold
                            f2 = (fold+1) % len(folds)
                            f3 = (fold+2) % len(folds)

                            parcial_name = base + '_' + word_embedding + '_' + str(epoca) + '_' + str(name_arq) + '_' + str(name_lista_grupos) + '_' + str(operador)
                            
                            df_train,df_test,nome_arq = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,True,parcial_name)

                            denisty_train, density_test, density_outlier = informacao_de_densidade(lista_grupos,df_train,df_test,df_outlier)

                            mvae, encoder, decoder = MultimodalVAE(arq,len(df_train[0]),len(lista_grupos),operador)

                            mvae.fit([df_train,denisty_train], [df_train,denisty_train], epochs=epoca, batch_size=32, verbose=0)

                            save_model(encoder,caminho + nome_arq + '_encoder')
                            save_model(decoder,caminho + nome_arq + '_decoder')

                            # para carregar = "load_model('path/to/location')""

                            X_train,_,_ = encoder.predict([df_train,denisty_train])
                            X_test,_,_ = encoder.predict([df_test,density_test])
                            X_outlier,_,_ = encoder.predict([df_outlier,density_outlier])

                            reps = (X_train,X_test,X_outlier)

                            lista_reps.append(reps)
                            
                            del df_train
                            del df_test
                            del X_train
                            del X_test
                            del X_outlier
                            collect()

                        
                        for modelo in tqdm(list(modelos.keys())):

                            metricas = init_metrics()
                                
                            for reps in lista_reps:    
                                start = time.time()
                                values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
                                end = time.time()
                                temp = end - start
                                
                                values['tempo'] = temp

                                save_values(metricas,values)

                            nome_arquivo = base + '_MVAE_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

                            parametros_linha = modelo + '_' + str(epoca) + '_' + str(arq) + '_' + str(lista_grupos) + '_' + str(operador)

                            escreve(metricas,nome_arquivo,parametros_linha,len(folds), caminho_resultado)
                        
                        del lista_reps
                        collect()                  

    del df_outlier
    collect()

  del df
  del df_int
  del df_out
  collect()

## TSNE MVAE-FakeNews

In [ ]:
dic = {
    'fcn' : { 3 : (50	,[384, 256]	,[3, 5, 7, 9],	'average'),
              5 : (50	,[256, 128],[3, 5, 7, 9],	'average'),
              7 : (10,	[384, 256],	[3, 5, 7, 9],	'average'),
              10 : (10,	[384, 256],	[2, 4, 6, 8, 10],	'average'),
    },
    'fnn' : { 3 : (50,[256, 128],	[2, 3, 4, 5, 6, 7, 8, 9, 10],	'multiply'),
              5 : (10,	[256, 128],	[2, 4, 6, 8, 10],	'multiply'),
              7 : (5,	[384, 256],	[2, 4, 6, 8, 10],	'multiply'),
              10 : (5,	[384, 256],	[2, 4, 6, 8, 10],	'multiply'),
    },
    'fakebr' : { 3 : (5	,[384, 256]	,[2, 4, 6, 8, 10],	'multiply'), 
              5 : (10	,[256],	[2, 3, 4, 5, 6, 7, 8, 9, 10],	'multiply'),
              7 : (10	,[256],	[2, 3, 4, 5, 6, 7, 8, 9, 10],	'multiply'),
              10 : (10,	[256],	[2, 3, 4, 5, 6, 7, 8, 9, 10],	'multiply'),
    },
}

In [ ]:
word_embedding = 'DistilBERT Multilingua'
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier = np.array(df_out[word_embedding].to_list())

  for percent in tqdm(percents):

    epoca = dic[base][percent][0]
    arq = dic[base][percent][1]
    operador = dic[base][percent][3]
    lista_grupos = dic[base][percent][2]

    for fold in tqdm(folds):

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)
      
      df_train,df_test = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,False)

      denisty_train, density_test, density_outlier = informacao_de_densidade(lista_grupos,df_train,df_test,df_outlier)

      mvae, encoder, decoder = MultimodalVAE(arq,len(df_train[0]),len(lista_grupos),operador)

      mvae.fit([df_train,denisty_train], [df_train,denisty_train], epochs=epoca, batch_size=32, verbose=0)

      X_train,_,_ = encoder.predict([df_train,denisty_train])
      X_test,_,_ = encoder.predict([df_test,density_test])
      X_outlier,_,_ = encoder.predict([df_outlier,density_outlier])

      name = 'MVAE/' + base + '/' + str(percent) + '/tsne_fold' + str(f1) 

      plot_TSNE(np.concatenate((X_train,X_test)), X_outlier, name)

## Evalaution MVAE-LIWK

In [ ]:
from tqdm.notebook import tqdm

epocas = [5,10,50]

arquiteturas = [[384,256],   # entrada tem tamanho do vetor da word-embedding
                [256,128],   # aqui são definidas a segunda camada e/ou a camada codificada
                [256]
                ] 

caminho_resultado = './resultados/'

percents = [3,5,7,10]

operadores = ['concatenate','multiply','average','subtract','add']

caminho = './modelos/MVAE_golo_mari/' # caminho para salvar os modelos de rede neurais aprendidos

word_embeddings = ['DistilBERT Multilingua']
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  for word_embedding in tqdm(word_embeddings):

    df_outlier_emb = np.array(df_out[word_embedding].to_list())
    df_outlier_fet = np.array(df_out['features_normalized'].to_list()).astype('float32')

    for percent in tqdm(percents):
    
        for epoca in tqdm(epocas):

            for arq in tqdm(arquiteturas):
                
                name_arq = str(arq).replace('[', '(')
                name_arq = name_arq.replace(']',')')

                for operador in tqdm(operadores):

                    lista_reps = []

                    for fold in tqdm(folds):

                        f1 = fold
                        f2 = (fold+1) % len(folds)
                        f3 = (fold+2) % len(folds)

                        parcial_name = base + '_' + word_embedding + '_' + str(epoca) + '_' + str(name_arq) + '_' + str(operador)
                        
                        df_train_emb,df_test_emb,nome_arq = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,True,parcial_name)
                        df_train_fet,df_test_fet = train_test_split_pipeline(df_int, percent,'features_normalized', f1,f2,f3,False)

                        df_train_fet = df_train_fet.astype('float32')
                        df_test_fet = df_test_fet.astype('float32')

                        mvae, encoder, decoder = MultimodalVAE(arq,len(df_train_emb[0]),len(df_train_fet[0]),operador)

                        mvae.fit([df_train_emb,df_train_fet], [df_train_emb,df_train_fet], epochs=epoca, batch_size=32, verbose=0)

                        save_model(encoder,caminho + nome_arq + '_encoder')
                        save_model(decoder,caminho + nome_arq + '_decoder')

                        # para carregar = "load_model('path/to/location')""

                        X_train,_,_ = encoder.predict([df_train_emb,df_train_fet])
                        X_test,_,_ = encoder.predict([df_test_emb,df_test_fet])
                        X_outlier,_,_ = encoder.predict([df_outlier_emb,df_outlier_fet])

                        reps = (X_train,X_test,X_outlier)

                        lista_reps.append(reps)

                        del df_train_emb
                        del df_test_emb
                        del df_train_fet
                        del df_test_fet
                        del X_train
                        del X_test
                        del X_outlier
                        collect()
                    
                    for modelo in tqdm(list(modelos.keys())):

                        metricas = init_metrics()
                            
                        for reps in lista_reps:

                            start = time.time()
                            values,fpr,tpr = evaluate_model(reps[0], reps[1], reps[2], modelos[modelo])
                            end = time.time()
                            temp = end - start
                            values['tempo'] = temp
                            save_values(metricas,values)

                        nome_arquivo = base + '_MVAE-GM_' + word_embedding + '_' + modelo.split('_')[0] + '_' + str(percent) + '.csv'

                        parametros_linha = modelo + '_' + str(epoca) + '_' + str(arq) + '_' + str(operador)

                        escreve(metricas,nome_arquivo,parametros_linha,len(folds), caminho_resultado)
                    del lista_reps
                    collect()

    del df_outlier_emb
    del df_outlier_fet
    collect()

  del df
  del df_int
  del df_out
  collect()

## TSNE MVAE-LIWK


In [ ]:
dic = {
    'fcn' : { 3 : (50	,	[256, 128],	'average'),
              5 : (50	,	[384, 256],	'average'),
              7 : (50	,	[384, 256],	'average'),
              10 : (50,		[384, 256],	'average'),
    },
    'fnn' : { 3 : (10	,	[256]	,'subtract'),
              5 : (10	,	[384, 256]	,'subtract'),
              7 : (50	,	[384, 256],	'add'),
              10 : (10,		[384, 256],	'subtract'),
    },
    'fakebr' : { 3 : (5	,	[384, 256],	'average'), 
              5 : (5	,	[384, 256]	,'average'),
              7 : (50	,	[384, 256],	'average'),
              10 : (50	,	[384, 256],	'average'),
    },
}

In [ ]:
word_embedding = 'DistilBERT Multilingua'
  
for base in tqdm(bases.keys()):
  df = bases[base]

  df_int = df[df['class'] == 1]

  df_out = df[df['class'] == -1]

  folds = df_int['fold'].unique()

  df_outlier_emb = np.array(df_out[word_embedding].to_list())
  df_outlier_fet = np.array(df_out['features_normalized'].to_list()).astype('float32')

  for percent in tqdm(percents):

    epoca = dic[base][percent][0]
    arq = dic[base][percent][1]
    operador = dic[base][percent][2]

    for fold in tqdm(folds):

      f1 = fold
      f2 = (fold+1) % len(folds)
      f3 = (fold+2) % len(folds)
      
      df_train_emb,df_test_emb = train_test_split_pipeline(df_int, percent,word_embedding, f1,f2,f3,False)
      df_train_fet,df_test_fet = train_test_split_pipeline(df_int, percent,'features_normalized', f1,f2,f3,False)

      df_train_fet = df_train_fet.astype('float32')
      df_test_fet = df_test_fet.astype('float32')

      mvae, encoder, decoder = MultimodalVAE(arq,len(df_train_emb[0]),len(df_train_fet[0]),operador)

      mvae.fit([df_train_emb,df_train_fet], [df_train_emb,df_train_fet], epochs=epoca, batch_size=32, verbose=0)

      X_train,_,_ = encoder.predict([df_train_emb,df_train_fet])
      X_test,_,_ = encoder.predict([df_test_emb,df_test_fet])
      X_outlier,_,_ = encoder.predict([df_outlier_emb,df_outlier_fet])


      name = 'MVAE-LIWK/' + base + '/' + str(percent) + '/tsne_fold' + str(f1) 

      plot_TSNE(np.concatenate((X_train,X_test)), X_outlier, name)